In [ ]:
#######################################################################
# Copyright (C)                                                       #
# 2016-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

# Policy Evaluation

In [ ]:
import itertools
from progress.bar import Bar
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.table import Table
matplotlib.use('Agg')

---

## Problem setup

- **States**: boxes in a rectangular grid
- **Immediate reward at time $t$**: $r_t=-1$
- **Terminal states (where we would like the agent to be as fast as possible)**: 
    - in terminal states, $r_t=0$
    - the box on the top left corner and the box on the bottom right corner
- **Policy to under which we will evaluate the values of states**: 
    - a random policy, $\pi(s,a)=0.25$, for all action $a$ available in state $s$ ($a \in A(s)$)
    - in this example, $a \in A(s) = \{\uparrow, \downarrow, \leftarrow, \rightarrow\}$ for $s \in S+$
- **Rules / Environment**:
    - Deterministic:
        - $p(s', r|s, a)=1$ if and only
            - agent can use $a$ to move from $s$ to $s'$, which implies that
                - $s'$ must be right next to and touching $s$, since $a$ only moves the agent for one step in the horizontal or the vertical direction
        - otherwise $p(s', r|s, a)=0$

### Environment hyper-parameters and functions

In [ ]:
WORLD_SIZE = 10

In [ ]:
def is_terminal(state):
    x, y = state
    return (x == 0 and y == 0) or (x == WORLD_SIZE - 1 and y == WORLD_SIZE - 1)

def is_trap(state):
    x, y = state
    return (x == 0 and y == 2) or (x == 1 and y == 2) or (x == 1 and y == 3) or (x == 2 and y == 3) or (x == 2 and y == 2) or (x == 2 and y == 1) or (x == 1 and y == 2) or (x == 2 and y == 0)

def is_gold(state):
    x, y = state
    return (x == 1 and y == 1) or (x == 0 and y == 1) or (x == 1 and y == 0)

def is_wall(state):
    i, j = state
    if i == 2 and j in list(range(4, 8)):
        return True
    elif i in list(range(3, 7)) and j == 2:
        return True
    elif i in list(range(3, 6)) and j == 7:
        return True
    elif i == 7 and j in list(range(2, 8)):
        return True

This is how the environment responses to an agent's action when the agent is in some given state.
- GridWorld is a deterministic environment, that is, $p(s', r|s, a)$ is either 1 (if s' in accessible in s AND reachable by a) or 0 (if s' is not accessible in s OR not reachable by a) for any 4-tuple (s', r, s, a).
- Therefore, s' and r can be directly determined from s and a.

In [ ]:
def step(state, action):
    """Given a tuple of state and action, output a tuple of immediate reward and next state"""
    
    # check if the agent has already reached the terminal states; if yes, keep it there
    if is_terminal(state):
        return state, 0

    next_state = (np.array(state) + action).tolist()
    x, y = next_state

    # check if the agent is outside the grid; if yes, put it back to its last position in the grid
    if x < 0 or x > WORLD_SIZE - 1 or y < 0 or y > WORLD_SIZE - 1 or is_wall(next_state):
        next_state = state
            
    if is_trap(next_state):
        reward = -2
    else:
        reward = -1

    return next_state, reward

### Agent actions

In [ ]:
# left, up, right, down
ACTIONS = [np.array([0, -1]), # left
           np.array([-1, 0]), # up
           np.array([0, 1]),  # right
           np.array([1, 0])]  # down
ACTION_PROB = [25/100, 25/100, 25/100, 25/100]  # modification

In [ ]:
ACTIONS

In [ ]:
for i, a in enumerate(ACTIONS):
    if np.array_equal(a, np.array([0, -1])):
        print(i)

---

## Policy Iteration Algorithm

**Motivation**

The optimal value function is a result of the optimal behavior of the agent:
$$v_{*}(s)=\max_{\pi}v_{\pi}(s)=v_{\text{argmax}_{\pi}v_{\pi}}=v_{\pi_*} \text{ for all } s \in S+$$

Yet the agent can only behave optimally when it is guided by the optimal value function:
$$\pi_{*}(s)=\text{argmax}_{a} \sum_{s', r}p(s', r|s, a)v_{*}(s') \text{ for all } s \in S+$$

This inter-dependence is similar for the responsibilities and parameters when using EM to optimize for a GMM. We can see that the formula for responsibilities involve the parameters and vice versa. The reason why the EM algorithm works is because one iteration of the E step and the M step is guaranteed to improve the log-likehood, until a local or global maximum.

Similarly, we can prove that one iteration of policy evaluation and policy improvement (together called policy iteration) is guaranteed to improve the value function and therefore the policy. Furthermore, we can show that the value function stops improving if only if it becomes equal to the optimal value function. Therefore, policy iteration guarantees not only the monotonic increase of the value function but also its convergence to a global maximum (a property that EM for GMM does not have).

<img src='em1.png' width=400>
<img src='em2.png' width=400>

### Iterative Policy Evaluation Algorithm, for estimating $V=v_{\pi}$, essential for Policy Improvement Algorithm
Without an accurate estimation of value function of the previous policy, it makes no sense to act greedily with respect to it and hope that we find a better policy. Similarly, in EM for GMM, it makes no sense to update the parameters without first computing the responsibilities.

In [ ]:
def eval_policy(policy=None, first_round=True, niters=None, in_place=True, discount=1.0):
    
    # "intialize V(s) = 0, for all s in S+"
    new_state_values = np.zeros((WORLD_SIZE, WORLD_SIZE))  
    
    # do not play active role, just for recording the number of iterations before convergence
    iteration = 0
    
    theta = 1e-4
    while True:  # first loop ("Loop:")
    
        if in_place:
            # when new_state_values get updated in line 29, it is immediately used
            state_values = new_state_values
        else:
            # when new_state_values get updated in line 29, it is NOT immediately used
            state_values = new_state_values.copy()
            
        old_state_values = state_values.copy() # new values

        # second loop ("Loop for each s in S:")
        # compute new values iteratively and put them in a grid
        for i, j in itertools.product(range(WORLD_SIZE), range(WORLD_SIZE)):
            if not is_wall((i, j)):
                # everything here corresponds to "V(s) <- ..."
                value = 0
                if first_round:
                    for k, action in enumerate(ACTIONS):
                        (next_i, next_j), reward = step([i, j], action)
                        value += ACTION_PROB[k] * (reward + discount * state_values[next_i, next_j])
                else:
                    action = policy[i][j]
                    (next_i, next_j), reward = step([i, j], action)
                    value = 1 * (reward + discount * state_values[next_i, next_j])
                new_state_values[i, j] = value

        # instead of iteratively computing deltas and comparing them
        # first compute elementwise difference between the grid containing new values from the grid containing old values
        # then apply max to get the maximum delta value
        # corespond to "delta <- 0", "v <- V(s)" and "delta <- max(delta, |v-V(s)|)"
        max_delta_value = abs(old_state_values - new_state_values).max()
        
        # do not play active role, just for recording the number of iterations before convergence
        iteration += 1
        
        # stop if precision requirement is met
        if niters is None:
            if max_delta_value < theta:
                break
        else:
            if iteration == niters:
                break
    
    return new_state_values, iteration

### Policy Improvement Algorithm

In [ ]:
def improve_policy(values):
    new_policy = np.zeros_like(values)
    new_policy_indices = np.zeros_like(values)
    new_policy = new_policy.tolist()
    for i in range(values.shape[0]):
        for j in range(values.shape[1]):
            if not is_wall((i, j)):
            
                value_here = values[i, j]

                if j-1 >= 0 and not is_wall((i, j-1)):
                    value_left = values[i, j-1]
                else:
                    value_left = value_here

                if i+1 <= WORLD_SIZE-1 and not is_wall((i+1, j)):
                    value_down = values[i+1, j]
                else:
                    value_down = value_here

                if j+1 <= WORLD_SIZE-1 and not is_wall((i, j+1)):    
                    value_right = values[i, j+1]
                else:
                    value_right = value_here

                if i-1 >= 0 and not is_wall((i-1, j)):
                    value_up = values[i-1, j]
                else:
                    value_up = value_here

                immediate_rewards = np.array([step((i, j), a)[1] for a in ACTIONS]) 
                surrounding_values = np.array([value_left, value_up, value_right, value_down])
                
                action_index = np.argmax(immediate_rewards + surrounding_values)
                action_here = ACTIONS[action_index]

                new_policy[i][j] = action_here
                new_policy_indices[i, j] = action_index
    return new_policy, new_policy_indices

## Visualization

In [ ]:
def draw_image(image):
    fig, ax = plt.subplots()
    ax.set_axis_off()
    tb = Table(ax, bbox=[0, 0, 1, 1])

    nrows, ncols = image.shape
    width, height = 1.0 / ncols, 1.0 / nrows

    # Add cells
    for (i, j), val in np.ndenumerate(image):
        tb.add_cell(i, j, width, height, text=val,
                    loc='center', facecolor='white')

        # Row and column labels...
    for i in range(len(image)):
        tb.add_cell(i, -1, width, height, text=i+1, loc='right',
                    edgecolor='none', facecolor='none')
        tb.add_cell(-1, i, width, height/2, text=i+1, loc='center',
                    edgecolor='none', facecolor='none')
    ax.add_table(tb)

def figure_4_1(eval_policy_only=False):
    # While the author suggests using in-place iterative policy evaluation,
    # Figure 4.1 actually uses out-of-place version.
    #_, asycn_iteration = eval_policy(in_place=True)
    total_iter = 0
    
    if eval_policy_only:
        values, sync_iter = eval_policy(policy=None, first_round=True, niters=1000, in_place=False)
        total_iter = sync_iter
        policy=None
    else:
        first_round=True
        policy=None
        for i in range(20):
            print(f'Policy Iteration - Round {i}')
            values, sync_iter = eval_policy(policy=policy, first_round=first_round, niters=50, in_place=False)
            total_iter += sync_iter
            policy, _ = improve_policy(values); first_round = False
        draw_image(np.round(values, decimals=2))
    
    #print('In-place: {} iterations'.format(asycn_iteration))
    print('Synchronous: {} iterations'.format(total_iter))

    plt.savefig('figure_4_1.png')
    plt.close()
    
    return values, policy

In [ ]:
opt_values, opt_policy = figure_4_1(eval_policy_only=True)

In [ ]:
opt_values, opt_policy = figure_4_1(eval_policy_only=False)

Motivated by: https://cs.stanford.edu/people/karpathy/reinforcejs/gridworld_dp.html

In [ ]:
def plot_values(values):
    fig, ax = plt.subplots(figsize=(7, 7))
    for i in range(values.shape[0]):
        for j in range(values.shape[1]):
            if is_wall((i, j)):
                values[i, j]=np.nan
    im = ax.matshow(values)
    for (i, j), z in np.ndenumerate(values):
        if is_trap((i, j)):
            ax.text(j, i, 'TRAP\n(r=-2)\n{:0.2f}'.format(z), ha='center', va='center', color='red')
        elif is_wall((i, j)):
            ax.text(j, i, '', ha='center', va='center')
        elif values[i, j] < -9:
            ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center', color='white')
        else:
            ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_values(opt_values)

## Monte Carlo Methods for Policy Iteration

Tasks to do (2019.10.20):
- give `eval_policy_mc` a new name (done)
- go through code, change var names (done)
- go through code, add comments, including how they relate to the current chapter (done)

In [ ]:
def init_action_probas(epsilon):
    
    vs = np.random.uniform(size=(WORLD_SIZE, WORLD_SIZE)) 
    _, greedy_action_indices = improve_policy(vs)
    
    action_probas = np.zeros((10, 10, 4))  # think of this as a cake ...
    for (row, col), greedy_action_index in np.ndenumerate(greedy_action_indices):
        action_probas[row, col] = epsilon / 4
        action_probas[row, col, int(greedy_action_index)] = 1 - epsilon + epsilon / 4
        
    return action_probas

In [ ]:
def update_action_probas(action_probas, qsa, s_t, epsilon):
    greedy_action_index = np.argmax(qsa[s_t])
    action_probas[s_t[0], s_t[1]] = epsilon / 4
    action_probas[s_t[0], s_t[1], greedy_action_index] = 1 - epsilon + epsilon / 4
    return action_probas

In [ ]:
def get_states_actions():
    S = [(i, j) for i, j in itertools.product(range(WORLD_SIZE), range(WORLD_SIZE)) if not is_wall((i, j))]
    S = np.array(S)
    A = np.array(ACTIONS)
    return S, A

In [ ]:
def get_action_from_state(state, actions, action_probas):
    action_index = np.random.choice([0, 1, 2, 3], p=normalize_probas(action_probas[state[0], state[1]]))
    action = actions[action_index]
    return action, action_index

In [ ]:
def normalize_probas(probas):
    return probas / probas.sum()

In [ ]:
qsa = np.zeros((WORLD_SIZE, WORLD_SIZE, len(ACTIONS)))

In [ ]:
def evaluate_qsa_monteCarlo(num_traversals=50000, 
                            epsilon=0.1, 
                            decreasing_epsilon=False,
                            optimal_v=None,
                            method='mc'):

    ########## Initialization ##########
    
    qsa = np.zeros((WORLD_SIZE, WORLD_SIZE, len(ACTIONS)))   # initialize value for each state-action pair
    action_probas = init_action_probas(epsilon)  # connects q(s, a) and policy
    
    S, A = get_states_actions()
    
    # initialize a sum of returns and a counter of times sampled for each state-action pair
    total_returns = np.zeros((WORLD_SIZE, WORLD_SIZE, len(ACTIONS)))
    total_occurrences = np.zeros((WORLD_SIZE, WORLD_SIZE, len(ACTIONS)))
    
    # not included in the algorithm, only used for collecting stats
    precisions = []

    ##########
    
    # randomly picking the starting state-action pair for each traversal
    a_indices = np.random.randint(len(A), size=num_traversals)
    As = A[a_indices]
    
    # TRAVERSAL LOOP
    eps = epsilon
    if method == 'mc' and epsilon < 0.5: print('epsilon is too small, might take too long to run...')
    for n in log_progress(list(range(num_traversals)), every=100, name='No. Traversals Attempted'):
            
        # MC/Sarsa: initialize S
        while True:
            s_index = np.random.choice(len(S))
            s = S[s_index]
            x, y = tuple(s)
            if not is_wall(s) and not (x <= 4 and y <= 4): break
    
        # MC: obtain the starting state-action pair for this traversal
        # Sarsa: choose A from S using policy derived from Q (e.g., epsilon-greedy) [but does not matter too much here]
        a = As[n]
        action_index = a_indices[n]
        
        if method == 'mc':  # first-visit method
        
            # one traversal / one sampling round
            episode_a_indices, episode_sa, episode_r = [], [], []
            while True:

                episode_sa.append((tuple(s), tuple(a))); episode_a_indices.append(action_index)  # record state-action pair
                s, r = step(s, a); episode_r.append(r) # move and record immediate reward
                action_index = np.random.choice([0, 1, 2, 3], p=normalize_probas(action_probas[s[0], s[1]]))  # pick a new action probabilistically
                a = A[action_index]
                
                if is_terminal(s): break

            # by definition, the value of a state-action pair is the expected cumulative reward the agent can gain by starting from it
            # now, in Monte Carlo, we simple replace the "expected cumulative reward" with "empirical mean of cumulative reward"

            cumulative_reward = 0
            for t in reversed(range(len(episode_r))):
                
                cumulative_reward = cumulative_reward + episode_r[t]  # this cumulative reward can be interpreted differently depending on when we enter the if statement below

                if episode_sa[t] not in episode_sa[:t]:
                    
                    (s_t, a_t), action_index = episode_sa[t], episode_a_indices[t] # first visit of state-action pair (s_t, a_t)

                    # this following step cannot be written as gamma * total_return[(*s_t, i)] + cumulative_reward
                    # may have to do with the convergence of Monte Carlo Methods in general

                    sa_pair_index = (*s_t, action_index)
                    total_returns[sa_pair_index] += cumulative_reward; total_occurrences[sa_pair_index] += 1
                    qsa[sa_pair_index] =  total_returns[sa_pair_index] / total_occurrences[sa_pair_index]
                    action_probas = update_action_probas(action_probas, qsa, s_t, eps)
                    
        elif method == 'sarsa':
            
            # hyperparameters
            alpha = 0.5
            lamb = 1.
            
            s = tuple(s)  # initialize S
            a, a_index = get_action_from_state(s, A, action_probas)  # choose A from S using policy derived from Q (at the begining)
            
            depth = 0
            
            while True:
                
                if is_terminal(s): break
                
                s_prime, r = step(s, a); s_prime = tuple(s_prime)  # take action, observe S', R
                a_prime, a_prime_index = get_action_from_state(s_prime, A, action_probas)  # choose A' from S' using policy dervied from Q
                
                qsa[(*s, a_index)] += \
                    alpha * (r + lamb * qsa[(*s_prime, a_prime_index)] - qsa[(*s, a_index)])
                action_probas = update_action_probas(action_probas, qsa, s, eps)
                s = s_prime; a = a_prime
                
                depth += 1
    
        if decreasing_epsilon:
            eps = epsilon / (n+1)
        
        if n % 1000 == 0 and optimal_v is not None:
            qsa_max = np.max(qsa, axis=2)
            errors = abs(qsa_max-optimal_v)
            precision = np.max(errors[~np.isnan(errors)])
            precisions.append(precision)
        
    return qsa, action_probas, precisions

### $\epsilon$-soft Methods / On-policy methods (will explain later what this means)

Recall $\epsilon$-greedy methods for solving multi-armed bandits problems, in which non-greedy actions are explored with a probability of $\epsilon$. The same thing is happening here, except that now we have many states instead of just one states. Acting greedily now means acting greedily with respect to the value function (the cumulative reward into the future) instead of just immediate rewards.

In [ ]:
qsa_5, action_probas_5, precisions_5 = evaluate_qsa_monteCarlo(num_traversals=50000, epsilon=0.5, decreasing_epsilon=True, optimal_v=opt_values)

In [ ]:
qsa_sarsa, action_probas_sarsa, precisions_sarsa = \
    evaluate_qsa_monteCarlo(num_traversals=50000, 
                            epsilon=0.1, 
                            decreasing_epsilon=False, 
                            optimal_v=opt_values, 
                            method='sarsa')

In [ ]:
qsa_7, action_probas_7, precisions_7 = evaluate_qsa_monteCarlo(num_traversals=50000, epsilon=0.7, decreasing_epsilon=True, optimal_v=opt_values)

In [ ]:
qsa_10, action_probas_10, precisions_10 = evaluate_qsa_monteCarlo(num_traversals=50000, epsilon=1, decreasing_epsilon=True, optimal_v=opt_values)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.arange(0, 50000, 1000), precisions_5, label='$\epsilon=0.5$ with 1/n decay')
# plt.plot(np.arange(0, 50000, 1000), precisions_7, label='$\epsilon=0.7$ with 1/n decay')
# plt.plot(np.arange(0, 50000, 1000), precisions_10, label='$\epsilon=1.0$ with 1/n decay')
# plt.plot(np.arange(0, 50000, 1000), precisions_sarsa, label='sarsa')
plt.ylabel('Max Error', fontsize=15); plt.xlabel('Iterations of Monte Carlo', fontsize=15)
plt.title('Precision of Estimation over Time', fontsize=15)
plt.legend(fontsize=15)
plt.grid()
plt.show()

In [ ]:
def plot_values_with_action_and_errors(qsa, action_probas, opt_policy):
    
    print('4-by-4 square on the top left corner is not accessible as start states.')
    
    qsa_max = np.max(qsa, axis=2)
    optimal_action_indices = action_probas.argmax(axis=2)
    
    num_errors = 0
    errors = np.zeros_like(optimal_action_indices)
    for i, row in enumerate(opt_policy):
        for j, policy in enumerate(row):
            if not is_wall((i, j)):
                if tuple(policy) != tuple(ACTIONS[optimal_action_indices[i, j]]):
                    errors[i, j] = 1
                    num_errors += 1

    for i in range(qsa_max.shape[0]):
            for j in range(qsa_max.shape[1]):
                if is_wall((i, j)):
                    qsa_max[i, j]=np.nan

    fig, ax = plt.subplots(figsize=(7, 7))
    im = ax.matshow(qsa_max)

    actions_names = ["left", "up", "right", "down"]

    for (i, j), action_index in np.ndenumerate(optimal_action_indices):
        action_name = actions_names[action_index]
        if is_wall((i, j)):
            ax.text(j, i, '', ha='center', va='center', color='black')
        elif is_terminal((i, j)):
            ax.text(j, i, str(round(qsa_max[i, j], 2)), ha='center', va='center', color='black', fontsize=15)
        else:
            if errors[i, j] == 0:
                if qsa_max[i, j] >= -9:
                    text_color = 'black'
                else:
                    text_color = 'white'    

            else:
                text_color = 'red'
            if is_trap((i, j)):
                ax.text(j, i, f'{round(qsa_max[i, j], 2)}\n{action_name}\nTRAP', ha='center', va='center', color=text_color, fontsize=13)
            else:
                ax.text(j, i, f'{round(qsa_max[i, j], 2)}\n{action_name}', ha='center', va='center', color=text_color, fontsize=13)


    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print('Total number of errors:', num_errors)

In [ ]:
plot_values_with_action_and_errors(qsa_5, action_probas_5, opt_policy)

In [ ]:
plot_values_with_action_and_errors(qsa_7, action_probas_7, opt_policy)

In [ ]:
plot_values_with_action_and_errors(qsa_10, action_probas_10, opt_policy)

**Conclusion**

- In the book, there is a quote related to this, that is,
"All learning control methods face a dilemma: They seek to learn action values conditional on subsequent optimal behavior, but they need to behave non-optimally in order to explore all actions (to ﬁnd the optimal actions)." 

- This dilemma will be solved by off-policy methods, which use two policies, one that is learned about and that becomes the optimal policy, and one that is more exploratory and is used to generate behavior.

- In contrast to off-policy methods, $\epsilon$-soft methods are called on-policy methods, which means that the policy that is learned about and the one that explores are the same policy (a special case of off-policy methods).